In [6]:
import pandas as pd
import sys, os, glob
# from tqdm import tqdm, tqdm_notebook
import tqdm

import requests

In [7]:
def get_completion(prompt, n_predict=1000):
    url = 'http://localhost:3030/completion'
    headers = {'Content-Type': 'application/json'}
    data = {
        'prompt': prompt,
        'n_predict': n_predict
    }
    response = requests.post(url, headers=headers, json=data)

    if response.status_code == 200:
        return response.json()
    else:
        return None  # You can handle error cases as needed

In [33]:
import os

def list_files_with_contents(base_path, file_type):
    file_dict = {}
    
    for root, _, files in os.walk(base_path):
        for file in files:
            if file.endswith(file_type):
                file_path = os.path.join(root, file)
                with open(file_path, 'r') as f:
                    file_contents = f.read()
                file_dict[os.path.join(root, file)] = file_contents
    
    return file_dict

# Example usage:
base_path = "THE-BOT"
file_type = ".py"
result = list_files_with_contents(base_path, file_type)
print(result.keys())

dict_keys(['THE-BOT/runserver.py', 'THE-BOT/test/flsk-login/connect.py', 'THE-BOT/test/flsk-login/app.py', 'THE-BOT/test/aiml_test/bot.py', 'THE-BOT/app/__init__.py', 'THE-BOT/app/views.py', 'THE-BOT/app/modules/__init__.py', 'THE-BOT/app/modules/weather/__init__.py', 'THE-BOT/app/modules/weather/ywthr.py', 'THE-BOT/app/modules/weather/get_weather_method.py', 'THE-BOT/app/modules/map_distance/__init__.py', 'THE-BOT/app/modules/map_distance/lat_lon.py', 'THE-BOT/app/modules/map_distance/dist.py', 'THE-BOT/app/modules/viki/__init__.py', 'THE-BOT/app/modules/viki/en_wiki.py', 'THE-BOT/app/modules/viki/.ipynb_checkpoints/en_wiki-checkpoint.py'])


In [48]:
def make_prompt(code):
    a = f"""###USER : Describe the below code line by line, describing each method and variable and what they do. Also describe the logic, breakdown of the code line by line in bullet points
Code : 
```
{code}
```

### PyDocs: 
"""
    return a

def write_content_to_md_file(filename, content):
    directory = os.path.dirname(filename)
    
    if not os.path.exists(directory):
        os.makedirs(directory)

    with open(filename, 'w') as md_file:
        md_file.write(content)

In [55]:
docs = {}
for files in tqdm.notebook.tqdm(result.keys()):
    try:
        prompt = make_prompt(result[files])
        docs = get_completion(prompt, n_predict=1000)
        file_name = f"docs/{files}"
        file_name = file_name.split(".")[0] + ".md"
        write_content_to_md_file(file_name, docs['content'])
    except:
        print(f"failed for file : {files}")
        continue

  0%|          | 0/16 [00:00<?, ?it/s]